**1) Import**

In [53]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import random
import re

In [54]:
from google.colab import files
uploaded=files.upload()

Saving Test_1.csv to Test_1 (2).csv


In [101]:
df=pd.read_csv("Test_1.csv")
df.head()

,review_id,title,year,user_review
0,1603,Counter-Strike: Global Offensive,2015,"Nice graphics, new maps, weapons and models. B..."
1,1604,Counter-Strike: Global Offensive,2018,I would not recommend getting into this at its...
2,1605,Counter-Strike: Global Offensive,2018,Edit 11/12/18I have tried playing CS:GO recent...
3,1606,Counter-Strike: Global Offensive,2015,The game is great. But the community is the wo...
4,1607,Counter-Strike: Global Offensive,2015,I thank TrulyRazor for buying this for me a lo...


In [102]:
df1=df.drop(labels=['review_id','year'], axis=1)
df1.head()

,title,user_review
0,Counter-Strike: Global Offensive,"Nice graphics, new maps, weapons and models. B..."
1,Counter-Strike: Global Offensive,I would not recommend getting into this at its...
2,Counter-Strike: Global Offensive,Edit 11/12/18I have tried playing CS:GO recent...
3,Counter-Strike: Global Offensive,The game is great. But the community is the wo...
4,Counter-Strike: Global Offensive,I thank TrulyRazor for buying this for me a lo...


**2) Tokenization and Removing Stop words**

---



In [103]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')
import nltk
nltk.download('omw-1.4')
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [108]:
def text_prepare(input_text, ar_text):
    out_text = delete_links(input_text)
    out_text = delete_repeated_characters(out_text)
    out_text = clean_text(out_text)
    out_text = delete_stopwords(out_text)
    out_text = out_text.lower()
    return out_text
def delete_links(input_text):
    pettern  = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    out_text = re.sub(pettern, ' ', input_text)
    return out_text
def delete_repeated_characters(input_text):
    pattern  = r'(.)\1{2,}'
    out_text = re.sub(pattern, r"\1\1", input_text)
    return out_text
def clean_text(input_text):
    replace = r'[/(){}\[\]|@âÂ,;\?\'\"\*…؟–’،!&\+-:؛-]'
    out_text = re.sub(replace, " ", input_text)
    words = nltk.word_tokenize(out_text)
    words = [word for word in words if word.isalpha()]
    out_text = ' '.join(words)
    return out_text
def delete_stopwords(input_text):
  from nltk.corpus import stopwords
  stop_words = set(stopwords.words("english"))
  tokenizer = nltk.tokenize.WhitespaceTokenizer()
  tokens = tokenizer.tokenize(input_text)
  wnl = nltk.WordNetLemmatizer()
  lemmatizedTokens =[wnl.lemmatize(t) for t in tokens]
  out_text = [w for w in lemmatizedTokens if not w.lower() in stop_words]
  out_text = ' '.join(out_text)
  return out_text

In [109]:
df1['cleaned']=df1['user_review'].apply(text_prepare, args=(False,))

In [110]:
df1.head()

,title,user_review,cleaned,tokenized
0,Counter-Strike: Global Offensive,"Nice graphics, new maps, weapons and models. B...",nice graphic new map weapon model developer li...,"[nice, graphic, new, map, weapon, model, but, ..."
1,Counter-Strike: Global Offensive,I would not recommend getting into this at its...,would recommend getting current state csgo ha ...,"[i, would, recommend, getting, current, state,..."
2,Counter-Strike: Global Offensive,Edit 11/12/18I have tried playing CS:GO recent...,edit tried playing cs go recently ha dramatica...,"[edit, i, tried, playing, cs, go, recently, ha..."
3,Counter-Strike: Global Offensive,The game is great. But the community is the wo...,game great community worst match russian maybe...,"[the, game, great, but, community, worst, if, ..."
4,Counter-Strike: Global Offensive,I thank TrulyRazor for buying this for me a lo...,thank trulyrazor buying long time ago insisted...,"[i, thank, trulyrazor, buying, long, time, ago..."


In [111]:
def tokenize(txt):
  tokens=re.split('\W+',txt)
  return tokens
df1['tokenized']=df1['cleaned'].apply(lambda x: tokenize(x.lower()))
df2=df1.drop(labels=['user_review'], axis=1)
df2.head()

,title,cleaned,tokenized
0,Counter-Strike: Global Offensive,nice graphic new map weapon model developer li...,"[nice, graphic, new, map, weapon, model, devel..."
1,Counter-Strike: Global Offensive,would recommend getting current state csgo ha ...,"[would, recommend, getting, current, state, cs..."
2,Counter-Strike: Global Offensive,edit tried playing cs go recently ha dramatica...,"[edit, tried, playing, cs, go, recently, ha, d..."
3,Counter-Strike: Global Offensive,game great community worst match russian maybe...,"[game, great, community, worst, match, russian..."
4,Counter-Strike: Global Offensive,thank trulyrazor buying long time ago insisted...,"[thank, trulyrazor, buying, long, time, ago, i..."


**3) Stemming**

In [113]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer() 
for w in df1['tokenized'][2]:
    print(w, "", ps.stem(w))

edit  edit
tried  tri
playing  play
cs  cs
go  go
recently  recent
ha  ha
dramatically  dramat
gotten  gotten
worse  wors
player  player
openly  openli
obviously  obvious
cheating  cheat
even  even
saying  say
chat  chat
wall  wall
aim  aim
people  peopl
clean  clean
spraying  spray
wall  wall
even  even
dm  dm
casual  casual
far  far
beyond  beyond
used  use
know  know
many  mani
cheater  cheater
game  game
begin  begin
since  sinc
cs  cs
go  go
f  f
p  p
playground  playground
hacker  hacker
valve  valv
care  care
never  never
buying  buy
another  anoth
valve  valv
game  game
dont  dont
care  care
game  game
player  player
want  want
money  money
originali  originali
played  play
cs  cs
go  go
since  sinc
release  releas
enjoyed  enjoy
time  time
sadly  sadli
im  im
uninstalling  uninstal
forever  forev
many  mani
hacker  hacker
smurfs  smurf
yes  ye
even  even
prime  prime
hacker  hacker
careful  care
yes  ye
lot  lot
even  even
try  tri
hide  hide
wall  wall
aim  aim
getting  get
h

**4)TF idf tfidf**

In [118]:
first_sentence = df1['cleaned'][1]
second_sentence = df1['cleaned'][2]
#split so each word have their own string
first_sentence = first_sentence.split(" ")
second_sentence = second_sentence.split(" ")#join them to remove common duplicate words
total= set(first_sentence).union(set(second_sentence))
print(total)

{'forcing', 'bugs', 'wall', 'hacking', 'lifeguards', 'wish', 'sometimes', 'cs', 'past', 'lackluster', 'last', 'money', 'one', 'getting', 'putting', 'f', 'played', 'phone', 'little', 'many', 'lack', 'overwatch', 'mother', 'gone', 'buy', 'day', 'cheater', 'death', 'want', 'find', 'headshot', 'seems', 'sadly', 'esea', 'originali', 'dm', 'brand', 'generator', 'flex', 'current', 'prevent', 'recently', 'update', 'smurfs', 'think', 'far', 'work', 'hide', 'ha', 'another', 'get', 'p', 'go', 'skill', 'bottom', 'account', 'care', 'fun', 'threats', 'doe', 'throwing', 'buying', 'cancer', 'aim', 'pointless', 'dreaded', 'banned', 'may', 'global', 'changing', 'simply', 'hassle', 'made', 'le', 'key', 'recommend', 'real', 'try', 'toxic', 'uninstalling', 'badly', 'tried', 'anti', 'hackers', 'losing', 'dont', 'betting', 'regretting', 'gotten', 'would', 'smoke', 'seige', 'point', 'see', 'promise', 'gameplay', 'implemented', 'guess', 'spraying', 'banning', 'livehttps', 'careful', 'obviously', 'dmg', 'look',

In [119]:
wordDictA = dict.fromkeys(total, 0) 
wordDictB = dict.fromkeys(total, 0)
for word in first_sentence:
    wordDictA[word]+=1
    
for word in second_sentence:
    wordDictB[word]+=1

In [120]:
pd.DataFrame([wordDictA, wordDictB])

,forcing,bugs,wall,hacking,lifeguards,wish,sometimes,cs,past,lackluster,...,killing,problem,game,cheating,insult,temporary,combat,edit,put,every
0,0,1,0,0,0,1,0,0,1,1,...,0,1,5,3,0,0,2,0,1,1
1,1,0,4,1,1,0,1,5,0,0,...,1,0,8,2,2,1,0,1,0,0


In [121]:
def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return(tfDict)
#running our sentences through the tf function:
tfFirst = computeTF(wordDictA, first_sentence)
tfSecond = computeTF(wordDictB, second_sentence)
#Converting to dataframe for visualization
tf = pd.DataFrame([tfFirst, tfSecond])

In [122]:
print (tf)

    forcing      bugs      wall   hacking  lifeguards      wish  sometimes  \
0  0.000000  0.010989  0.000000  0.000000    0.000000  0.010989   0.000000   
1  0.003623  0.000000  0.014493  0.003623    0.003623  0.000000   0.003623   

         cs      past  lackluster  ...   killing   problem      game  \
0  0.000000  0.010989    0.010989  ...  0.000000  0.010989  0.054945   
1  0.018116  0.000000    0.000000  ...  0.003623  0.000000  0.028986   

   cheating    insult  temporary    combat      edit       put     every  
0  0.032967  0.000000   0.000000  0.021978  0.000000  0.010989  0.010989  
1  0.007246  0.007246   0.003623  0.000000  0.003623  0.000000  0.000000  

[2 rows x 237 columns]


In [124]:
import math
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    print(idfDict)
    for word, val in idfDict.items():
        print(word,val)
        idfDict[word] = math.log10(N / (float(val) + 1))
        
    return(idfDict)
#inputing our sentences in the log file
idfs = computeIDF([wordDictA, wordDictB])
print(idfs)

{'forcing': 0, 'bugs': 0, 'wall': 0, 'hacking': 0, 'lifeguards': 0, 'wish': 0, 'sometimes': 0, 'cs': 0, 'past': 0, 'lackluster': 0, 'last': 0, 'money': 0, 'one': 0, 'getting': 0, 'putting': 0, 'f': 0, 'played': 0, 'phone': 0, 'little': 0, 'many': 0, 'lack': 0, 'overwatch': 0, 'mother': 0, 'gone': 0, 'buy': 0, 'day': 0, 'cheater': 0, 'death': 0, 'want': 0, 'find': 0, 'headshot': 0, 'seems': 0, 'sadly': 0, 'esea': 0, 'originali': 0, 'dm': 0, 'brand': 0, 'generator': 0, 'flex': 0, 'current': 0, 'prevent': 0, 'recently': 0, 'update': 0, 'smurfs': 0, 'think': 0, 'far': 0, 'work': 0, 'hide': 0, 'ha': 0, 'another': 0, 'get': 0, 'p': 0, 'go': 0, 'skill': 0, 'bottom': 0, 'account': 0, 'care': 0, 'fun': 0, 'threats': 0, 'doe': 0, 'throwing': 0, 'buying': 0, 'cancer': 0, 'aim': 0, 'pointless': 0, 'dreaded': 0, 'banned': 0, 'may': 0, 'global': 0, 'changing': 0, 'simply': 0, 'hassle': 0, 'made': 0, 'le': 0, 'key': 0, 'recommend': 0, 'real': 0, 'try': 0, 'toxic': 0, 'uninstalling': 0, 'badly': 0, 't

In [125]:
tfidf=(tf/idfs)
print(tfidf)

    forcing      bugs      wall   hacking  lifeguards      wish  sometimes  \
0  0.000000  0.036505  0.000000  0.000000    0.000000  0.036505   0.000000   
1  0.012036  0.000000  0.048144  0.012036    0.012036  0.000000   0.012036   

        cs      past  lackluster  ...   killing   problem      game  cheating  \
0  0.00000  0.036505    0.036505  ...  0.000000  0.036505  0.182524  0.109514   
1  0.06018  0.000000    0.000000  ...  0.012036  0.000000  0.096288  0.024072   

     insult  temporary    combat      edit       put     every  
0  0.000000   0.000000  0.073009  0.000000  0.036505  0.036505  
1  0.024072   0.012036  0.000000  0.012036  0.000000  0.000000  

[2 rows x 237 columns]
